# Exercise 1

Use the functions below to generate an animation of radar images from a single day.

In [ ]:
import collections.abc
import urllib.request
import io
import re
import zipfile
import PIL
import PIL.Image
import numpy as np

TIF_FILE = re.compile(".*.tif")

def tif_to_rain_rate(image):
    """
    Transform tif image to rain rates according to the formula given
    at https://opendata.smhi.se/apidocs/radar/data.html.
    
    Args:
        image(PIL.Image.Image) The .tif images containing the radar data.
        
    Returns:
        numpy array containing the converted rain rates.
    """
    data = np.array(image).astype(np.float32)
    data[data >= 255] = np.nan
    dbz = data * 0.4 - 30.0
    rain_rate = (10 ** (dbz / 10.0) / 200.0) ** (1.0 / 1.5)
    rain_rate[dbz < 5] = 0.0
    return rain_rate

class SMHIRadarImages(collections.abc.Iterable):
    """
    Provides access to all SMHI radar images for a given day.
    
    Iterating over an SMHIRadarImages object will yield the radar data converted
    to rain rates.
    
    Attributes:
        file(``zipfile.ZipFile``): The zipfile object containing the images.
        image_files: The list of filenames of the image files in the zipfile.
        n_images(``int``): The number of images of the day.
    """
    def __init__(self, year, month, day):
        """
        Create SMHIRadarImages object for given date.
        """
        self.year = year
        self.month = month
        self.day = day
        url = (f"https://opendata-download-radar.smhi.se/api/version/latest"
               f"/area/sweden/product/comp/{year}/{month}/{day}.zip?format=tif")
        response = urllib.request.urlopen(url)
        stream = io.BytesIO(response.read())
        self.file = zipfile.ZipFile(stream)
        self.image_files = [f for f in self.file.namelist() if TIF_FILE.match(f)]
        self.n_images = len(self.image_files)

    def __iter__(self):
        for filename in self.image_files:
            data = io.BytesIO(self.file.read(filename))
            image = PIL.Image.open(data)
            yield tif_to_rain_rate(image)
            
    def __repr__(self):
        return f"SMHIRadarImages({self.year}, {self.month}, {self.day})"

In [ ]:
def average_images(smhi_images, n_frames=8):
    """
    Bins images from one day into n_frames temporal bins and calculates
    the averages over each bin.
    
    Args:
        smhi_images(SMHIImages): SMHIImages object providing access to the
             rain rates for a given day.
        n_frames(int): Into how many frames to bin the data for the given day.
    Returns:
        3-dimensional numpy.ndarray containing the different frames along the
        first axis and the radar composite along the following two.
    """
    n_images = smhi_images.n_images
    result = np.zeros((n_frames, 887 // 2, 471 // 2))
    images_per_bin = n_images // n_frames 
    for i, rain_rates in enumerate(smhi_images):
        index = min(i // images_per_bin, n_frames - 1)
        result[index, :, :] += rain_rates[:-1:2, :-1:2]
    result /= images_per_bin
    return result

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.colors import LogNorm

def create_animation(data):
    """
    Creates an animation from a 3D array of rain rate data.

    Args:
        data(numpy.ndarray): 3D array containing different images along the
             first axis and the image dimensions alsong the second and third.
    
    Returns:
        matplotlib.animation.ArtisAnimation object containing the animated radar
        images.
    """
    plt.rcParams["animation.html"] = "jshtml"
    fig = plt.figure(figsize=(5, 8))
    
    frames = []
    for i in range(data.shape[0]):
        img = plt.imshow(np.maximum(data[i], 1e-3), animated=True,
                          cmap="magma", norm=LogNorm(1e-2, 1e2))
        frames.append([img])
    plt.colorbar(img, label="Rain rate [mm/h]")
    plt.close()
    return animation.ArtistAnimation(fig,
                                     frames,
                                     interval=50,
                                     repeat_delay=1000,
                                     blit=True)

# Exercise 2

Create a conda environment from the environment ``ssdp.yml`` environment file provided in this folder.

# Exercise 3

Parallelize the calculation of the animation frames using ``ipyparallel``. The function below may be handy for this.

> **Note**: Don't try to parallelize the generation of the animation. This probably won't work.

In [ ]:
import calendar

import itertools

def list_days(year, start_month, end_month):
    """
    Creates an iterator of the days in a range of months.
    
    Args:
        year(int): The year.
        start_month(int): The first returned day will correspond
            to the first day of this month.
        end(int): The last returned day will correspond
            to the last day of this month.
            
    Return:
        An iterator over tuples (year, month, day) corresponding to each
        day in the specified range.
    """
    days = [[(year, month, day + 1) for day in range(calendar.monthrange(year, month)[1])]
            for month in range(start_month, end_month + 1)]
    return list(itertools.chain.from_iterable(days))
    

# Exercise 4

Parallelize the calculation using either Dasks's ``delayed`` method or the ``Bag`` class.

> **Note**: You may have to restart your notebook to free up your memory.

# Exercise 5

Modify the ``SMHIRadarImages`` class so that it can be lazily loaded into a Dask array. Then use Dask to calculate the minimum, maximum, mean and standard deviation of each pixel along the first dimension.

> **Note**: Assume that each day has 288 images. If this is not the case simply return the last images
> for indices larger than the number of available images.